In [1]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import fetch_20newsgroups
from scipy.special import gammaln

In [2]:
#################################################################################################################
#
# Train data must be of the form
# <document_index> <word_index in the vocabulary> <token count>
#
# where document_index and word_index start at 1
#################################################################################################################
class LDA_gibbs_sampler:
    
    def __init__(self, num_topics, alpha=1, eta=0.1, num_iterations=200, save_samples_sched = None):
        self.K = num_topics
        self.alpha = alpha
        self.eta = eta
        self.num_iterations=num_iterations
        self.count_vectorizer = CountVectorizer(stop_words='english', min_df=2, max_df=0.95)
        self.save_samples_sched = save_samples_sched
        
    #Return log likelihood for p(w|z) and p(z)
    def compute_log_likelihood(self, C_wt, C_dt):
        subtractor_w = gammaln(C_wt.sum(axis=0) + len(self.vocab)*self.eta).sum() # for log p(w|z)
        subtractor_d = gammaln(C_dt.sum(axis=1) + self.K*self.alpha).sum() # for log p(z)
    
        return gammaln(C_wt + self.eta).sum() - subtractor_w, gammaln(C_dt + self.alpha).sum() - subtractor_d            
       
    #Return count matrices
    # C_wt (i,j): number of times word i was assigned to topic j
    # C_dt (i,j): number of times a word in doc i was assigned to topic j
    def calculate_counts(self, sample):
        C_wt_z = np.zeros((len(self.vocab), self.K))
        C_dt_z = np.zeros((self.num_docs, self.K))

        #Populate counts given z
        for i in range(sample.shape[0]): #number of docs
            for j in range(sample.shape[1]): #number of words in vocab
                C_wt_z[j, sample[i,j]]+=1
                C_dt_z[i, sample[i,j]]+=1
                
        return C_wt_z, C_dt_z        
        
    def train(self, traindata, num_docs, vocab, verbose=False):
                       
        self.vocab = vocab

        self.num_docs = num_docs
        size_vocab = len(self.vocab)

        #Per G&S(2004)
        self.C_wt = np.zeros((size_vocab, self.K))
        self.C_dt = np.zeros((self.num_docs, self.K))

        #Topic-word assignments: Z[i,j] = k => topic k is responsible for word token j in doc i
        #Random init
        self.Z = np.random.randint(self.K, size=(num_docs, size_vocab))
        
        if (self.save_samples_sched is not None):
            self.samples = np.zeros((len(self.save_samples_sched), num_docs, size_vocab))
            sample_idx = 0
    
        for doc_idx, word_idx, token_count in traindata.astype(int):
            #the bag of words file starts at 1
            doc_idx-=1
            word_idx-=1

            topic_assignment = self.Z[doc_idx, word_idx]
            self.C_wt[word_idx, topic_assignment]+=token_count
            self.C_dt[doc_idx, topic_assignment]+=token_count          
        
        iteration = 0
        
        size_vocab_times_eta = (size_vocab*self.eta)
        K_times_alpha = (self.K*self.alpha)
        p = np.zeros(self.K)

        while iteration < self.num_iterations:            
                        
            if (verbose):
                print np.sum(self.compute_log_likelihood(self.C_wt, self.C_dt))
                
            if (iteration%10 == 0 and verbose):
                print iteration                           

            for doc_idx, word_idx, token_count in traindata.astype(int):
                #the bag of words file starts at 1
                doc_idx-=1
                word_idx-=1

                topic_assignment = self.Z[doc_idx, word_idx]
                #For each word token,the count matrices CWT and CDT are first decremented by one for the 
                #entries that correspond to the current topic assignment
                self.C_wt[word_idx, topic_assignment]-=token_count
                self.C_dt[doc_idx, topic_assignment]-=token_count
                #Then, a new topic is sampled from the distribution in Equation 3 and the count matrices CWT and CDT
                #are incremented with the new topic assignment                
                
                sum_dt_all_topics = np.sum(self.C_dt[doc_idx, :])

                for k in range(self.K):      
                    denominator = (np.sum(self.C_wt[:,k]) + size_vocab_times_eta)*(sum_dt_all_topics + K_times_alpha)
                    p[k] = (self.C_wt[word_idx, k] + self.eta)*(self.C_dt[doc_idx, k] + self.alpha)/denominator                  
                    
                p = p/np.sum(p)   

                #argmax will return the topic selected
                new_topic = np.argmax(np.random.multinomial(1, p))

                self.Z[doc_idx, word_idx] = new_topic
                #and the count matrices CWT and CDT are incremented with the new topic assignment
                self.C_wt[word_idx, new_topic]+=token_count
                self.C_dt[doc_idx, new_topic]+=token_count

            iteration+=1
            
            if (self.save_samples_sched is not None and iteration in self.save_samples_sched):
                self.samples[sample_idx] = self.Z
                sample_idx+=1                                

    #C_dt: matrix (num_docs x num_topics) where C_dt(d, j)=number of times a word from doc d has been assigned to topic j
    def compute_theta(self, C_dt):
        #Infer distribution of topics per document
        to_return = np.zeros(C_dt.shape)

        for d in range(C_dt.shape[0]):
            to_return[d]= [ (C_dt[d, j] + self.alpha)/(np.sum(C_dt[d, :]) + (self.K*self.alpha)) for j in range(self.K) ]

        return to_return
            
    #C_wt: matrix (size_vocab x num_topics) where C_wt(w, j)=number of times word w has been assigned to topic j        
    def compute_beta(self, C_wt):
        #Infer distribution of words per topic
        size_vocab = C_wt.shape[0]
        to_return = np.zeros((self.K, size_vocab))    

        for k in range(self.K):
            to_return[k]= [ (C_wt[w,k] + self.eta)/(np.sum(C_wt[:, k]) + (size_vocab*self.eta)) for w in range(size_vocab)]
    
        return to_return
    
    def print_topic_words(self, num_words, beta):
        feature_names = self.vocab
        
        for topic_id, topic in enumerate(beta):
            sorted_topic = topic.argsort()[:-num_words-1:-1]
            print "Topic %d: " % topic_id  + ", ".join([feature_names[i] for i in sorted_topic]) 


In [4]:
#Since our gibbs sampler will iterate through all word tokens in the corpus,
#test that after using scikit-learn's tokenizer, we are passing the data as expected
data_samples = [
    'weather: warm, cold, freezing, hot, windy, windy',
    'weather: dry, windy, moist, cold, etc',
    'freezing means dry and windy',
    'sports game, be it basketball, hockey or soccer, I feel better',
    'sports can be soothing. hockey, but I like soccer and basketball'
]

tf_vectorizer = CountVectorizer(stop_words='english')
tf = tf_vectorizer.fit_transform(data_samples)

# for i in range(len(data_samples)):
#     print "Document {}".format(i)
#     print tf.data[tf.indptr[i]:tf.indptr[i+1]]
#     print tf.indices[tf.indptr[i]:tf.indptr[i+1]]

dummy_data =  np.ndarray(shape=(tf.data.shape[0], 3), dtype=int)
doc_idx = 0
    
# print "Doc idx | word idx | token count"
for idx, token_count in enumerate(tf.data):
    if idx >= tf.indptr[doc_idx+1]:
        doc_idx+=1
    
    word_idx = tf.indices[idx]
    dummy_data[idx] = np.array([doc_idx+1, word_idx+1, token_count])

#Test on dummy data
keylist = tf_vectorizer.vocabulary_.keys()
keylist.sort()
    
lda_gibbs_sampler = LDA_gibbs_sampler(2, alpha=1, eta=0.01, num_iterations=20000)
lda_gibbs_sampler.train(traindata=dummy_data, num_docs=5, vocab=keylist)
theta = lda_gibbs_sampler.compute_theta(lda_gibbs_sampler.C_dt)
beta = lda_gibbs_sampler.compute_beta(lda_gibbs_sampler.C_wt)
lda_gibbs_sampler.print_topic_words(5, beta)

Topic 0: basketball, sports, soccer, hockey, like
Topic 1: windy, cold, dry, weather, freezing


In [5]:
#To run on the KOS blog data set
text_file = open('vocab.kos.txt', "r")
vocab = text_file.read().splitlines()
kos_data=np.loadtxt('docword.kos.txt')

In [6]:
#Change the num_iterations parameter to run for at least 1000 iterations
lda_gibbs_sampler = LDA_gibbs_sampler(10, alpha=5, eta=0.1, num_iterations=10)
lda_gibbs_sampler.train(traindata=kos_data, num_docs=3430, vocab=vocab)

In [ ]:
#To run on the 20newsgroups dataset
dataset_20ng = fetch_20newsgroups(subset='train', shuffle=True, random_state=1,
                             remove=('headers', 'footers', 'quotes'))
num_docs_to_load = 20

tf_20ng = tf_vectorizer.fit_transform(dataset_20ng.data[:num_docs_to_load])